In [1]:
import numpy as np
from get_data import data, raw_data
#Paso Inicial: Inicializo valores
k = 5 # Revisar
p = len(raw_data)
countries = raw_data.values[:,0]
n = data.shape[1]

def set_init_weights():
  w = []
  for _ in range(0,p):
    wi = np.random.rand(n)
    w.append(wi)
  return np.array(w)

def update_neighborhood_weight(radius, w_k):
  #neu - neuk ?
  umbral = np.sum(data[w_k])
  res = []
  for i in range(len(data)):
    if np.abs(np.sum(data[i]) - umbral) < radius:
      res.append(i)
  return np.array(res)

def kohonen(init_radius = np.sqrt(2), init_learn_rate = 0.5, max_epochs = 100):
  #Paso Inicial
  weights = set_init_weights()
  radius = init_radius
  learn_rate = init_learn_rate
  t = 1
  cut = False
  #Paso t
  while t < max_epochs and not cut:
    #Paso 1: Selecciono un registro de entrada Xp
    x_index = np.random.choice(range(data.shape[0]))
    x = data[x_index]
    #Paso 2: Encontrar la neurona ganadora
    # Xp se refiere a todo el conjunto o a un registro elegido de Xp?
    aux = []
    for w in weights:
      aux.append(np.sum(np.abs(data - w))) # PREGUNTAR si data o x
    aux = np.array(aux)
    w_k = np.argmin(aux)
    #Paso 3: Actualizar los pesos de las neuronas vecinas
    n_k = update_neighborhood_weight(radius,w_k)
    for j in range(len(weights)):
      if (j in n_k):
        weights[j] = weights[j] + learn_rate * (data[j]-weights[j])
    t += 1
  
  print(weights)
    
kohonen()

[[-5.07835215e-01  6.83900422e-01  1.14446814e-01  5.70778263e-01
  -1.02434720e+00 -1.76788945e-01 -1.24552737e+00]
 [-8.35987244e-01  4.17061387e-01  1.14446814e-01  4.87755971e-01
  -3.88952393e-01 -1.15927177e-01 -5.92441859e-01]
 [-2.88629490e-02 -2.29975778e-01  5.19166891e-01 -4.57921954e-01
   1.01954482e+00 -8.42977899e-01  2.02432458e-01]
 [-6.75636105e-01 -9.73310428e-01 -7.59510673e-01 -6.80942460e-01
   9.96208287e-01 -4.20236016e-01  1.69335744e+00]
 [-1.61860087e-01 -1.53014658e-01 -3.71414806e-01 -5.44076401e-02
   9.65684757e-02  4.01705694e-03  2.25532236e-01]
 [-7.58684535e-01  3.60884748e-01 -3.95361720e-01  2.09950606e-01
  -3.88952393e-01  2.49243430e-01 -8.31906548e-01]
 [-7.45556731e-01 -8.04780511e-01  1.20689367e+00 -1.45049525e+00
   5.00600337e-01 -1.55632235e+00  5.61342549e-01]
 [ 9.46022004e-01  3.26425602e-01  7.97488944e-02  4.11819438e-01
   4.96101988e-01 -6.23573736e-02 -3.40212219e-01]
 [ 1.17251750e+00  4.38127626e-01 -6.13851092e-01  6.60186887e-0